In [1]:
import sys
import numpy as np
import math
import operator
import functools
import json
from pprint import pprint

In [2]:
class Command:
    def __init__(self, on, xyz, offset=0):
        self.on = 1 if on == "on" else 0
        self.x, self.y, self.z = [[int(x)+offset for x in i[2:].split("..")] for i in xyz.split(",")]
    
    def updateCubes(self, cuboid):
        if self.x[0] >= 0 and self.x[1] <= 100 and self.y[0] >= 0 and self.y[1] <= 100 and self.z[0] >= 0 and self.z[1] <= 100:
            cuboid[self.x[0]:self.x[1]+1,self.y[0]:self.y[1]+1,self.z[0]:self.z[1]+1] = self.on
    
    def __repr__(self):
        return f"{self.on}: {self.x}:{self.x[1]-self.x[0]}, {self.y}:{self.y[1]-self.y[0]}, {self.z}:{self.z[1]-self.z[0]}"
        

In [3]:
def part1(arr):
    cubes = np.zeros((101, 101, 101))
    commands = [Command(*i.split(), 50) for i in arr]
    for c in commands:
        # print(c)
        c.updateCubes(cubes)
        # print(len(np.where(cubes == 1)[0]))
    
    return len(np.where(cubes == 1)[0])

In [71]:
class Cuboid:
    def __or__(self, other):
        return UnionCuboid(self, other)
    
    def __and__(self, other):
        return self.inter(other)
    
    def __sub__(self, other):
        return DiffCuboid(self, other)
    
class SimpleCuboid(Cuboid):
    xs: range
    ys: range
    zs: range
    
    def __init__(self, xs, ys, zs):
        self.xs = xs
        self.ys = ys
        self.zs = zs
    
    def __len__(self):
        return len(self.xs) * len(self.ys) * len(self.zs)
    
    def inter(self, other):
        if type(other) == SimpleCuboid:
            x_range = range(max(self.xs.start, other.xs.start), min(self.xs.stop, other.xs.stop))
            y_range = range(max(self.ys.start, other.ys.start), min(self.ys.stop, other.ys.stop))
            z_range = range(max(self.zs.start, other.zs.start), min(self.zs.stop, other.zs.stop))
            if any(len(r) == 0 for r in (x_range, y_range, z_range)):
                return NullCuboid
            return SimpleCuboid(x_range, y_range, z_range)
        else:
            return other.inter(self)
        
    def __repr__(self):
        return f"{{\"SimpleCuboid\": [{self.xs},{self.ys},{self.zs}]}}"
        
class _NullCuboid(Cuboid):
    instance = None

    def __new__(cls):
        if cls.instance is None:
            cls.instance = super(_NullCuboid, cls).__new__(cls)
        return cls.instance
    
    def __len__(self):
        return 0
    
    def inter(self, other):
        return NullCuboid
    
    def __repr__(self):
        return f"{{\"NullCuboid\": 0}}"

NullCuboid = _NullCuboid()
        
    
class UnionCuboid(Cuboid):
    left: Cuboid
    right: Cuboid
    
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
    def __len__(self):
        return len(self.left) + len(self.right) - len(self.left & self.right)
    
    def inter(self, other):
        left_int = self.left & other
        right_int = self.right & other

        if left_int is NullCuboid:
            return right_int
        if right_int is NullCuboid:
            return left_int
        return left_int | right_int
    
    def __repr__(self):
        return f"{{\"UnionCuboid\": [{self.left}, {self.right}]}}"
    
    
class DiffCuboid(Cuboid):
    main: Cuboid
    sub: Cuboid
    
    def __init__(self, main, sub):
        self.main = main
        self.sub = sub
    
    def __len__(self):
        return len(self.main) - len(self.main & self.sub)
    
    def inter(self, other):
        if (main_int := self.main & other) is NullCuboid:
            return NullCuboid
        return main_int - self.sub
    
    def __repr__(self):
        return f"{{ \"DiffCuboid\" : [{self.main}, {self.sub}]}}"
    
        
def part2(arr):
    commands = [Command(*i.split()) for i in arr]
    result = NullCuboid
    for c in commands:
        cuboid = SimpleCuboid(range(c.x[0],c.x[1]+1), range(c.y[0],c.y[1]+1), range(c.z[0],c.z[1]+1))
        if c.on:
            result |= cuboid
        else:
            result -= cuboid
    return len(result)

In [73]:
from IPython.core.display import display, HTML
display(HTML("<style>div.jp-OutputArea-output pre {white-space: pre;}</style>"))

if __name__ == "__main__":
    file = sys.argv[-1] if sys.argv[-1].endswith(".txt") else "input.txt"
    with open(file, "r") as f:
        arr = f.read().splitlines()

        print(part1(arr))
        print(part2(arr))

570915
1268313839428137
